In [1]:
!pip install -q ultralytics

In [2]:
import pandas as pd
from numpy.linalg import inv
import numpy as np
import math
import cv2
import torch
from matplotlib import pyplot as plt

import torch
from ultralytics import YOLO

In [3]:
model = YOLO('best.pt')  # Ensure you have the correct model file

# Ensure to use the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

In [4]:
def draw_prediction(img: np.ndarray,
                    class_name: str,
                    df: pd.core.series.Series,
                    color: tuple = (255, 0, 0)):
    '''
    Function to draw prediction around the bounding box identified by the YOLO
    The Function also displays the confidence score top of the bounding box
    '''

    cv2.rectangle(img, (int(df.xmin), int(df.ymin)),
                  (int(df.xmax), int(df.ymax)), color, 2)
    cv2.putText(img, class_name + " " + str(round(df.confidence, 2)),
                (int(df.xmin) - 10, int(df.ymin) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img

In [5]:
class KalmanFilter():
    def __init__(self,
                 xinit: int = 0,
                 yinit: int = 0,
                 fps: int = 30,
                 std_a: float = 0.001,
                 std_x: float = 0.0045,
                 std_y: float = 0.01,
                 cov: float = 100000) -> None:

        # State Matrix
        self.S = np.array([xinit, 0, 0, yinit, 0, 0])
        self.dt = 1 / fps

        # State Transition Model
        # Here, we assume that the model follow Newtonian Kinematics
        self.F = np.array([[1, self.dt, 0.5 * (self.dt * self.dt), 0, 0, 0],
                           [0, 1, self.dt, 0, 0, 0], [0, 0, 1, 0, 0, 0],
                           [0, 0, 0, 1, self.dt, 0.5 * self.dt * self.dt],
                           [0, 0, 0, 0, 1, self.dt], [0, 0, 0, 0, 0, 1]])

        self.std_a = std_a

        # Process Noise
        self.Q = np.array([
            [
                0.25 * self.dt * self.dt * self.dt * self.dt, 0.5 * self.dt *
                self.dt * self.dt, 0.5 * self.dt * self.dt, 0, 0, 0
            ],
            [
                0.5 * self.dt * self.dt * self.dt, self.dt * self.dt, self.dt,
                0, 0, 0
            ], [0.5 * self.dt * self.dt, self.dt, 1, 0, 0, 0],
            [
                0, 0, 0, 0.25 * self.dt * self.dt * self.dt * self.dt,
                0.5 * self.dt * self.dt * self.dt, 0.5 * self.dt * self.dt
            ],
            [
                0, 0, 0, 0.5 * self.dt * self.dt * self.dt, self.dt * self.dt,
                self.dt
            ], [0, 0, 0, 0.5 * self.dt * self.dt, self.dt, 1]
        ]) * self.std_a * self.std_a

        self.std_x = std_x
        self.std_y = std_y

        # Measurement Noise
        self.R = np.array([[self.std_x * self.std_x, 0],
                           [0, self.std_y * self.std_y]])

        self.cov = cov

        # Estimate Uncertainity
        self.P = np.array([[self.cov, 0, 0, 0, 0, 0],
                           [0, self.cov, 0, 0, 0, 0],
                           [0, 0, self.cov, 0, 0, 0],
                           [0, 0, 0, self.cov, 0, 0],
                           [0, 0, 0, 0, self.cov, 0],
                           [0, 0, 0, 0, 0, self.cov]])

        # Observation Matrix
        # Here, we are observing X & Y (0th index and 3rd Index)
        self.H = np.array([[1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]])

        self.I = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0],
                           [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])

        # Predicting the next state and estimate uncertainity
        self.S_pred = None
        self.P_pred = None

        # Kalman Gain
        self.K = None

        # Storing all the State, Kalman Gain and Estimate Uncertainity
        self.S_hist = [self.S]
        self.K_hist = []
        self.P_hist = [self.P]

    def pred_new_state(self):
        self.S_pred = self.F.dot(self.S)

    def pred_next_uncertainity(self):
        self.P_pred = self.F.dot(self.P).dot(self.F.T) + self.Q

    def get_Kalman_gain(self):
        self.K = self.P_pred.dot(self.H.T).dot(
            inv(self.H.dot(self.P_pred).dot(self.H.T) + self.R))
        self.K_hist.append(self.K)

    def state_correction(self, z):
        if z == [None, None]:
            self.S = self.S_pred
        else:
            self.S = self.S_pred + +self.K.dot(z - self.H.dot(self.S_pred))

        self.S_hist.append(self.S)

    def uncertainity_correction(self, z):
        if z != [None, None]:
            self.l1 = self.I - self.K.dot(self.H)
            self.P = self.l1.dot(self.P_pred).dot(self.l1.T) + self.K.dot(
                self.R).dot(self.K.T)
        self.P_hist.append(self.P)

def cost_fun(a, b):
    '''
    Cost function for filter Assignment
    Uses euclidean distance for choosing the filter
    '''

    sm = 0
    for i in range(len(a)):
        sm += (a[i] - b[i])**2
    return sm

In [12]:
import json
import numpy as np
import cv2
from collections import Counter


def group_similar_coordinates(coords, threshold=10):
    """
    Groups coordinates that are within a threshold distance from each other.
    Returns a list of unique coordinates (averaged within the group).
    Also updates the global frequency dictionary.
    """
    grouped_coords = []
    used = [False] * len(coords)

    for i in range(len(coords)):
        if used[i]:
            continue

        # Start a new group with the current coordinate
        current_group = [coords[i]]
        used[i] = True

        for j in range(i + 1, len(coords)):
            # Compute the Euclidean distance between coordinates
            dist = np.linalg.norm(np.array(coords[i]) - np.array(coords[j]))
            if dist < threshold:
                current_group.append(coords[j])
                used[j] = True

        # Average the grouped coordinates
        avg_coord = tuple(np.mean(current_group, axis=0))
        grouped_coords.append((avg_coord, len(current_group)))

        # Update global coordinate frequency
        if avg_coord in global_coord_frequency:
            global_coord_frequency[avg_coord] += len(current_group)
        else:
            global_coord_frequency[avg_coord] = len(current_group)

    return grouped_coords

def identify_stationary_objects(threshold=10):
    """
    Identifies stationary objects by grouping similar coordinates based on frequency of occurrence
    and merging those that are close together.
    """
    global global_coord_frequency, stationary_coords

    # Get the list of coordinates and their frequencies from the global frequency dictionary
    coords_with_freq = list(global_coord_frequency.items())

    def group_coordinates_by_proximity(coords_with_freq, threshold):
        """
        Groups coordinates in the global frequency dictionary by proximity and sums their frequencies.
        """
        grouped_freq = []
        used = [False] * len(coords_with_freq)

        for i in range(len(coords_with_freq)):
            if used[i]:
                continue

            current_group = [coords_with_freq[i][0]]  # Start a new group with the current coordinate
            total_freq = coords_with_freq[i][1]  # Initialize total frequency with current frequency
            used[i] = True

            for j in range(i + 1, len(coords_with_freq)):
                # Compute the Euclidean distance between the coordinates
                dist = np.linalg.norm(np.array(coords_with_freq[i][0]) - np.array(coords_with_freq[j][0]))
                if dist < threshold:
                    current_group.append(coords_with_freq[j][0])
                    total_freq += coords_with_freq[j][1]
                    used[j] = True

            # Average the grouped coordinates
            avg_coord = tuple(np.mean(current_group, axis=0))
            grouped_freq.append((avg_coord, total_freq))

        return grouped_freq

    # Group the coordinates by proximity
    grouped_coords_with_freq = group_coordinates_by_proximity(coords_with_freq, threshold)

    # Define a threshold for considering an object stationary based on frequency
    stationary_threshold = 25  # You can adjust this value

    # Find coordinates that occur above the threshold and add to stationary_coords
    stationary_coords = [(coord,freq) for coord, freq in grouped_coords_with_freq if freq > stationary_threshold]
    
    return stationary_coords

def is_close_to_blacklist(coord, black_list, threshold=1):
    for black_coord in black_list:
        distance = np.sqrt((coord[0] - black_coord[0])**2 + (coord[1] - black_coord[1])**2)
        if distance <= threshold:
            return True
    return False

In [13]:
import cv2
import json
import pandas as pd
from collections import deque, Counter
import numpy as np

number_of_past_frames = 5
def is_shuttle_in_rest(shuttle_coords_queue, number_of_past_frames, threshold=5):

    if len(shuttle_coords_queue) < number_of_past_frames:
        return False  # Not enough data to determine movement

    # Get the minimum and maximum x and y coordinates from the last 20 frames
    x_coords = [coord[0] for coord in shuttle_coords_queue]
    y_coords = [coord[1] for coord in shuttle_coords_queue]
    
    # Check if the difference between max and min is within the threshold
    if max(x_coords) - min(x_coords) <= threshold and max(y_coords) - min(y_coords) <= threshold:
        return True  # Shuttle is at rest
    else:
        return False  # Shuttle is moving

In [34]:

# Global variables to track coordinate frequencies and stationary object coordinates
global_coord_frequency = {}
stationary_coords = []  # List to store coordinates of detected stationary objects

def real_time_detection_and_tracking(video_path, output_json_path, black_list_provided = False):
    global global_coord_frequency, stationary_coords

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS: {fps}")

    ret, frame = cap.read()
    frame_height, frame_width = frame.shape[:2]
    out = cv2.VideoWriter('realtime_tracking_kalman.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Dictionary to store tracking data
    tracking_data = {}
    coord_counter = Counter()  # Directly maintain a counter for coordinates
    frame_count = 0
    shuttle_coords_queue = deque(maxlen=5) 

    if not black_list_provided:
        # # 12sec.mp4
        # black_list = [(2477.4579819544147, 1486.213537329001),(1529.9303159460558, 326.81500203630566), 
        #               (1285.9160690307617, 848.287064022488), (2700.828621018101, 594.6784457354479),
        #               (904.2800679524739, 558.4161139594185), (1285.8978812081473, 849.1556815011161), 
        #               (2701.013951455393, 595.3080670756679)]
        
        # # stop.mp4
        # black_list = [(904.2800679524739, 558.4161139594185),
        #               (1285.8978812081473, 849.1556815011161),
        #               (2701.013951455393, 595.3080670756679)]
        
        # drop.mp4
        black_list = [(639.6225493808962, 347.902106806917), (292.45753370012557, 139.51016199021112),
                     (36.80934792376579, 199.977830115785)]
        # black_list = []
        
    else:
        black_list = ext_black_list

    rest_coords = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run object detection on the current frame (replace with your model inference)
        results = model([frame])
        print(frame_count)
        # Assuming YOLO model results
        boxes = results[0].boxes.xyxy.cpu().numpy()  # Get the bounding boxes
        class_ids = results[0].boxes.cls.cpu().int().numpy()  # Get the class IDs
        scores = results[0].boxes.conf.cpu().numpy()  # Get the confidence scores

        # Convert detection results into a DataFrame
        df_current = pd.DataFrame({
            'xmin': boxes[:, 0],
            'ymin': boxes[:, 1],
            'xmax': boxes[:, 2],
            'ymax': boxes[:, 3],
            'class_id': class_ids,
            'confidence': scores
        })

        # Initialize a list to collect the current frame's coordinates for grouping
        current_coords = []


        for _, row in df_current.iterrows():
            coord = [(row['xmin'] + row['xmax']) / 2, (row['ymin'] + row['ymax']) / 2]  # Object center coordinates

            # Only track specific class (e.g., class_id == 0 for a ball/shuttlecock)
            if row['class_id'] == 0:
                if not is_close_to_blacklist(coord, black_list, threshold=20):
                    current_coords.append(coord)
                    # print(current_coords)
                    shuttle_coords_queue.append(coord)
                    
                    if is_shuttle_in_rest(shuttle_coords_queue, 5):
                        print("rest")
                        rest_coords.append(coord)
                        print(rest_coords)
                        
                    tracking_data[f"{frame_count}_{_}"] = {
                        'x_center': coord[0],
                        'y_center': coord[1]
                    }

        tmp_img = frame.copy()
        rest_coords = group_similar_coordinates(rest_coords, threshold=10)
        if rest_coords: rest_coords = [rest_coords[0][0]]
        i = False
        for coord in rest_coords:
            print(rest_coords)
            i = True
            # Text position
            text_position = (int(coord[0]), int(coord[1]) - 10)  # Offset slightly above the coord
            print(text_position)
            # Text parameters
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1
            thickness = 2  # Thicker for bold effect

            # Draw outline (black background for contrast)
            cv2.putText(tmp_img, 'Shuttle is at rest', text_position, font, font_scale, (0, 0, 0), thickness + 3)

            # Draw the actual text
            cv2.putText(tmp_img, 'Shuttle is at rest', text_position, font, font_scale, (255, 255, 255), thickness)  # White text for visibility
         
        # cv2.imwrite("tmp_img.jpg",tmp_img)   
        # if i: break
        group = group_similar_coordinates(current_coords)          
        coord_counter.clear()
        for coord, count in current_coords:
            coord_counter[coord] += count

        
        dummy = 10
        for x,y in current_coords:
            cv2.rectangle(tmp_img, (int(x)-dummy, int(y)-dummy),
                      (int(x)+dummy, int(y)+dummy), (255, 0, 100), 2)
            
        # dummy = 20
        # for x,y in black_list:
        #     cv2.rectangle(tmp_img, (int(x)-dummy, int(y)-dummy),
        #               (int(x)+dummy, int(y)+dummy), (0, 140, 255), 5)
            
        #     cv2.putText(tmp_img, 'Stationary', (int(x)-dummy, int(y)-dummy-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)

        cv2.imwrite("tmp_img.jpg",tmp_img)
        out.write(tmp_img)
        # if len(rest_coords)>1: break
        frame_count += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save tracking data to a JSON file
    with open(output_json_path, 'w') as json_file:
        json.dump(tracking_data, json_file, indent=4)

    # Identify stationary objects based on frequency
    stationary_coords = identify_stationary_objects()
    print("Stationary coordinates detected:")
    print(stationary_coords)
    if not black_list_provided:
        return stationary_coords, rest_coords
    
    else: return rest_coords

# Example usage
# real_time_detection_and_tracking("video.mp4", "tracking_data.json")


In [36]:
to_remove, rest = real_time_detection_and_tracking('drop.mp4', 'tracking_data.json')

# ext_black_list = []
# for coords,_ in to_remove:
#     ext_black_list.append(coords)

 
# print(ext_black_list)
# black_list = real_time_detection_and_tracking('drop.mp4', 'tracking_data.json', black_list_provided= True)


FPS: 24.93188276682772

0: 384x640 1 Shuttlecock, 492.9ms
Speed: 4.0ms preprocess, 492.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0

0: 384x640 2 Shuttlecocks, 547.3ms
Speed: 3.0ms preprocess, 547.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
1

0: 384x640 2 Shuttlecocks, 470.4ms
Speed: 4.0ms preprocess, 470.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
2

0: 384x640 2 Shuttlecocks, 488.4ms
Speed: 3.0ms preprocess, 488.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
3

0: 384x640 2 Shuttlecocks, 438.3ms
Speed: 3.0ms preprocess, 438.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
4

0: 384x640 2 Shuttlecocks, 522.3ms
Speed: 3.0ms preprocess, 522.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
5

0: 384x640 2 Shuttlecocks, 720.1ms
Speed: 5.0ms preprocess, 720.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
6

0: 384x640 2 Shuttlecocks

[(523.7864642889766, 187.89117134026), (518.8686879475912, 170.3552017211914), (510.091807774135, 194.94403621128626)]


In [199]:
def identify_stationary_objects(threshold=10):
    """
    Identifies stationary objects by grouping similar coordinates based on frequency of occurrence
    and merging those that are close together.
    """
    global global_coord_frequency, stationary_coords

    # Get the list of coordinates and their frequencies from the global frequency dictionary
    coords_with_freq = list(global_coord_frequency.items())

    def group_coordinates_by_proximity(coords_with_freq, threshold):
        """
        Groups coordinates in the global frequency dictionary by proximity and sums their frequencies.
        """
        grouped_freq = []
        used = [False] * len(coords_with_freq)

        for i in range(len(coords_with_freq)):
            if used[i]:
                continue

            current_group = [coords_with_freq[i][0]]  # Start a new group with the current coordinate
            total_freq = coords_with_freq[i][1]  # Initialize total frequency with current frequency
            used[i] = True

            for j in range(i + 1, len(coords_with_freq)):
                # Compute the Euclidean distance between the coordinates
                dist = np.linalg.norm(np.array(coords_with_freq[i][0]) - np.array(coords_with_freq[j][0]))
                if dist < threshold:
                    current_group.append(coords_with_freq[j][0])
                    total_freq += coords_with_freq[j][1]
                    used[j] = True

            # Average the grouped coordinates
            avg_coord = tuple(np.mean(current_group, axis=0))
            grouped_freq.append((avg_coord, total_freq))

        return grouped_freq

    # Group the coordinates by proximity
    grouped_coords_with_freq = group_coordinates_by_proximity(coords_with_freq, threshold)

    # Define a threshold for considering an object stationary based on frequency
    stationary_threshold = 5  # You can adjust this value

    # Find coordinates that occur above the threshold and add to stationary_coords
    stationary_coords = [(coord,freq) for coord, freq in grouped_coords_with_freq if freq > stationary_threshold]
    
    return stationary_coords
identify_stationary_objects()
print(stationary_coords)


[((83.21419034685407, 247.3387952532087), 8), ((349.14049835205077, 294.81441192626954), 10), ((350.7186596210186, 301.3290123572716), 13), ((530.0034423828125, 293.66875915527345), 10), ((537.1299947102865, 296.6829783121745), 6)]


In [194]:
global_coord_frequency

{(352.2607879638672, 183.4205780029297): 1,
 (85.07566452026367, 247.82427215576172): 1,
 (84.31482315063477, 247.66463470458984): 1,
 (83.4700698852539, 247.1454315185547): 2,
 (83.12549591064453, 247.32779693603516): 1,
 (82.73064804077148, 247.3534698486328): 1,
 (82.43071746826172, 246.9778060913086): 1,
 (81.35191345214844, 247.07815551757812): 1,
 (74.57586669921875, 246.18468475341797): 1,
 (74.56904220581055, 246.17891693115234): 1,
 (65.73158836364746, 245.2444839477539): 1,
 (65.72454643249512, 245.28805541992188): 1,
 (622.9013061523438, 429.5989074707031): 2,
 (599.8902893066406, 424.9875183105469): 1,
 (599.8878784179688, 424.9907684326172): 1,
 (619.0584411621094, 14.361498832702637): 1,
 (619.0563049316406, 14.353622436523438): 1,
 (623.9182739257812, 29.11090087890625): 1,
 (633.0013732910156, 67.05876922607422): 1,
 (633.0013122558594, 67.05677604675293): 1,
 (438.24346923828125, 286.0208282470703): 1,
 (426.2284851074219, 285.38623046875): 1,
 (426.2290802001953, 285.

In [115]:
import cv2
import numpy as np
import json

def load_tracking_data(json_file_path):
    """Load tracking data from the specified JSON file."""
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    # Extract court_info and net_info as lists of tuples
    court_info = [tuple(point) for point in data['court_info']]
    net_info = [tuple(point) for point in data['net_info']]
    
    court_info.sort(key=lambda x: x[1])
    net_info.sort(key=lambda x: x[1])
    
    return court_info, net_info

court_coords, net_coords = load_tracking_data('coordinates.json')

def is_shuttle_in_court(shuttle_coord, court_coords, net_coords):
    """Check if the shuttle is inside the court using cv2.pointPolygonTest."""
    # Convert court coordinates to a numpy array of shape (n, 1, 2) required by cv2
    # Court-1 (towards camera)
    court1 = court_coords[4:] + court_coords[2:4]
    court_polygon = np.array(court1, dtype=np.int32).reshape((-1, 1, 2))
    # Check if the point is inside the polygon
    result1 = cv2.pointPolygonTest(court_polygon, shuttle_coord, False)
    
    # Court-2 (away from camera)
    court2 = court_coords[:2] + court_coords[2:4]
    court_polygon = np.array(court2, dtype=np.int32).reshape((-1, 1, 2))
    # Check if the point is inside the polygon
    result2 = cv2.pointPolygonTest(court_polygon, shuttle_coord, False)
    
    '''
    1: shuttle in court near camera
    2: shuttle in court away from camera
    False: outside
    '''
    if result1 >= 0:
        return 1  # Shuttle is inside or on the court
    if result2 >= 0:
        return 2
    return False  # Shuttle is outside the court

# Example shuttle coordinate to test
shuttle_coordinate = (1000, 1200)  # Replace with actual shuttle coordinates

# Check if the shuttle is inside the court
print(is_shuttle_in_court(shuttle_coordinate, court_coords, net_coords))

1
